## Project: Maritime Fuels LCA
### Notebook 2: Running the baseline LCA calculations
#### This notebook is developed by Megan Roux and Massimo Pizzol, and includes adaptions of code provided by Massimo Pizzol, Nils Thonemann and Chris Mutel

This notebook relates to work done for the publication 'Consequential LCA of alternative maritime fuels' which can be found at: doi

Begin by importing all relevant packages (ensure you are in the right environment):

In [1]:
# Note: must be environment bw2
import brightway2 as bw
import pandas as pd
import numpy as np
import matplotlib
from lci_to_bw2 import * # this file has to be in the same folder as the notebook and raw data

Set the project (same as in Notebook 1):

In [2]:
bw.projects.set_current("fuels")

Now check which databases are in the project:

Should be ecoinvent 3.9 consequential, bioshphere3, Fuels_WtT, Fuels_TtW and Fuels_WtW. If not, go back to Notebook 1.

In [3]:
bw.databases

Databases dictionary with 23 objects, including:
	Fuels_db_SA1
	Fuels_db_SA1a
	Fuels_db_SA1b
	Fuels_db_SA1c
	Fuels_db_SA1d
	Fuels_db_SA1e
	Fuels_db_SA1f
	Fuels_db_SA2
	Fuels_db_SA2a
	Fuels_db_SA2b
Use `list(this object)` to get the complete list.

In [4]:
ei39db = bw.Database("ecoinvent 3.9 conseq")
biodb = bw.Database("biosphere3")

In [5]:
Fuels_WtT = [(act['database'], act['code']) for act in bw.Database('Fuels_db_WtT_MJ')]
Fuels_TtW = [(act['database'], act['code']) for act in bw.Database('Fuels_db_TtW_MJ')]
Fuels_WtW = [(act['database'], act['code']) for act in bw.Database('Fuels_db_WtW_MJ')]

Check the activities in your foreground database:

In [6]:
Fuels_WtT
# Here, all activities are included, even capital goods and eh2. 
# Pick any of the foreground databases

[('Fuels_db_WtT_MJ', 'VLSFO'),
 ('Fuels_db_WtT_MJ', 'bioMeOH'),
 ('Fuels_db_WtT_MJ', 'PO_slow'),
 ('Fuels_db_WtT_MJ', 'DAC'),
 ('Fuels_db_WtT_MJ', 'NH3_CCS'),
 ('Fuels_db_WtT_MJ', 'LNG'),
 ('Fuels_db_WtT_MJ', 'DME'),
 ('Fuels_db_WtT_MJ', 'Wood'),
 ('Fuels_db_WtT_MJ', 'eH2'),
 ('Fuels_db_WtT_MJ', 'eNH3'),
 ('Fuels_db_WtT_MJ', 'Straw'),
 ('Fuels_db_WtT_MJ', 'eMeOH_DOC'),
 ('Fuels_db_WtT_MJ', 'eMeOH_DAC'),
 ('Fuels_db_WtT_MJ', 'eMeOH_bio'),
 ('Fuels_db_WtT_MJ', 'PO_fast')]

We want to test that everything has worked, so we will do a quick LCA over one of the activities for one impact category.

First, choose an LCIA method (e.g. EF3.1 in the case of this study):

In [7]:
EF31 = [method for method in bw.methods if "EF v3.1" in str(method) 
        and "no LT" not in str(method)
        and "EN15804" not in str(method)]
# EF31

A test over one activity and one impact category to see if the LCA works:

In [9]:
# For eMeOH_DAC climate change impacts using EF31
mymethod = EF31[1]
print(mymethod)
Fuels_WtT[4]
myact = Fuels_WtT[4]
print(myact)
functional_unit = {myact: 1} 
lca = bw.LCA(functional_unit, mymethod)
lca.lci()
lca.lcia()
print(lca.score)

('EF v3.1', 'climate change', 'global warming potential (GWP100)')
('Fuels_db_WtT_MJ', 'NH3_CCS')
0.16148296038049706


The function to perform the LCA over all activities in all databases:

In [7]:
def dolcacalc(myact, mydemand, mymethod):
    my_fu = {myact: mydemand} 
    lca = bw.LCA(my_fu, mymethod)
    lca.lci()
    lca.lcia()
    return lca.score

# For WtT
def getLCAresults(list_acts, mymethod):
    
    all_activities = []
    results = []
    for a in list_acts:
        act = bw.Database(a[0]).get(a[1])
        all_activities.append(act['name'])
        results.append(dolcacalc(act,1,mymethod)) # 1 stays for one unit of each process
        #print(act['name'])
     
    results_dict = dict(zip(all_activities, results))
    
    return results_dict

Running the LCA calculations:

In [62]:
Results_WtT_EF31 = []
for m in EF31:
    results_all_fuels_WtT = getLCAresults(Fuels_WtT,m) # total impact per tech
    Results_WtT_EF31.append(results_all_fuels_WtT)

Results_TtW_EF31 = []
for m in EF31:
    results_all_fuels_TtW = getLCAresults(Fuels_TtW,m) # total impact per tech
    Results_TtW_EF31.append(results_all_fuels_TtW)

Results_WtW_EF31 = []
for m in EF31:
    results_all_fuels_WtW = getLCAresults(Fuels_WtW,m) # total impact per tech
    Results_WtW_EF31.append(results_all_fuels_WtW)

Making everything nice and pretty and exporting to Excel:

In [17]:
methods_names = []
for m in EF31:
    m_name = ' '.join(m)
    methods_names.append(m_name)

In [18]:
my_output = pd.DataFrame(Results_WtT_EF31, index=methods_names)
my_output.head()

,DME,VLSFO,Straw,NH3_CCS,eMeOH_bio,bioMeOH,eNH3,DAC,Wood,PO_slow,NH3_FGC,LNG,eH2,eMeOH_DAC,PO_fast,eMeOH_DOC
EF v3.1 acidification accumulated exceedance (AE),0.000941,0.000132,0.000315,0.000627,0.000526,0.000722,0.000418,0.000006,0.000693,0.001151,0.000723,0.000030,0.010953,0.000637,0.001058,-4853.079533
EF v3.1 climate change global warming potential (GWP100),0.171342,0.039751,0.141122,0.203900,0.092162,0.122900,0.078630,0.003847,0.299936,0.019203,0.161483,0.021443,2.085421,0.159885,0.051233,16824.718837
EF v3.1 climate change: biogenic global warming potential (GWP100),0.000123,0.000009,0.000015,0.000105,0.000066,0.000089,0.000053,-0.000004,0.000039,0.000037,0.000113,0.000001,0.001711,0.000053,0.000031,20.681994
EF v3.1 climate change: fossil global warming potential (GWP100),0.154453,0.039733,0.055665,0.203693,0.084569,0.108373,0.078397,0.003806,0.210086,-0.068842,0.161256,0.021438,2.076421,0.159643,0.035911,16762.571213
EF v3.1 climate change: land use and land use change global warming potential (GWP100),0.016766,0.000009,0.085442,0.000103,0.007528,0.014438,0.000180,0.000046,0.089811,0.088008,0.000114,0.000003,0.007289,0.000190,0.015290,41.465631


In [19]:
# Exporting to Excel
my_output.to_excel('Results_Fuels_WtT_MJ_EF31.xlsx')

In [66]:
my_output = pd.DataFrame(Results_TtW_EF31, index=methods_names)
my_output.head()

,LNG_combustion,DME_combustion,VLSFO_combustion,NH3_combustion,MeOH_combustion,PO_combustion
EF v3.1 acidification accumulated exceedance (AE),0.000462,0.002237,0.003426,0.000663,0.000619,0.000935
EF v3.1 climate change global warming potential (GWP100),0.133365,0.006839,0.167562,0.008902,0.005699,0.006602
EF v3.1 climate change: biogenic global warming potential (GWP100),0.000012,0.000369,0.000012,0.000012,0.000103,0.000087
EF v3.1 climate change: fossil global warming potential (GWP100),0.133344,0.006462,0.167542,0.008882,0.005588,0.006507
EF v3.1 climate change: land use and land use change global warming potential (GWP100),0.000008,0.000008,0.000008,0.000008,0.000008,0.000008


In [67]:
# Exporting to Excel
my_output.to_excel('Results_Fuels_TtW_MJ_EF31.xlsx')